In [73]:
from jinja2 import Environment, FileSystemLoader
import pandas as pd
import geopandas as gpd
pd.options.display.float_format = '{:,.2f}'.format
import plotly.express as px

In [74]:
pc = "Wroughton"

In [75]:
gdf = gpd.read_file("G:/OS_OpenData/OS_BoundaryLine/OS_BL_Parish_Wilts.shp")

In [76]:
# Create rank for PC area
gdf['AreaPC'] = gdf['HECTARES']
df_pc_sorted = gdf.sort_values(by = "AreaPC", ascending=False)
df_pc_sorted['rankArea'] = list(range(1, df_pc_sorted.shape[0]+1))

In [77]:
dfpc = df_pc_sorted[df_pc_sorted.NAME == pc]
pcAreaRank =dfpc['rankArea'].item()

In [ ]:
# Filter datasets to pc
df = pd.read_csv("data/outputData.csv")
df_pc = df[df.NAME == pc]
pcAreaHa = df_pc['AreaPC'].iloc[0].astype(int)


In [ ]:
# Select only items ranked higher than 10 for highlights boxes
df_pc_sorted = df_pc.sort_values(by = "rank", ascending=True)
df_pc_filter = df_pc_sorted[df_pc_sorted['rank']<=10]

# Select datasets you want to show in highlights boxes
df_pc_filter = df_pc_filter[df_pc_filter.datasetName.isin(['Long-Established Woodland and Wood Pasture','National Forest Inventory','Greenspace','PHI','WSBRC Priority Habitats'])]


In [ ]:
# Create dictionary to store all information for output
table_dict = {}
for category in df_pc.mapGroup:
    dataset_dict = {}
    df_pc_category =  df_pc[df_pc.mapGroup == category]
    
    for dataset in df_pc_category.datasetName:
        if df_pc_category.shape[0] >0:
            print(dataset)
            df_pc_filter_dataset = df_pc_category[df_pc_category.datasetName == dataset]
            # Check if dataset is an area, and if it is, include the value percent
            if df_pc_filter_dataset['unit'].iloc[0] == "Ha":
                df_small = df_pc_filter_dataset[['groupColumnValue','value', 'valuePercent']]
                
                df_small = df_small.rename(columns = {'groupColumnValue':df_pc_filter_dataset['groupColumnName'].iloc[0],
                                                    "value":df_pc_filter_dataset['unit'].iloc[0],
                                                    "valuePercent":"Percentage of PC"})
            # If dataset in metres or count, do not include valuePercent as is not relevant
            elif df_pc_filter_dataset['unitName'].iloc[0].lower() == "length" or df_pc_filter_dataset['unitName'].iloc[0].lower() == "count" :
                df_small = df_pc_filter_dataset[['groupColumnValue','value']]
                
                df_small = df_small.rename(columns = {'groupColumnValue':df_pc_filter_dataset['groupColumnName'].iloc[0],
                                                    "value":df_pc_filter_dataset['unit'].iloc[0]})
                
            # Check if column value has only one item, and if so, do not include figure as no point showing on graph
            if df_small[df_pc_filter_dataset['groupColumnName'].iloc[0]].unique().shape[0]==1:
                if df_pc_filter_dataset['groupColumnName'].iloc[0]== "Total":
                    dataset_dict[dataset] = {"table":"",
                                        "fig":"",
                                        "text": f"There is a total of {df_pc_filter_dataset['value'].values[0]} {df_pc_filter_dataset['unit'].values[0]} ",
                                        }
                else:
                    dataset_dict[dataset] = {"table":"",
                                        "fig":"",
                                        "text":f"There is {df_pc_filter_dataset['value'].values[0]} {df_pc_filter_dataset['unit'].values[0]} of  {df_pc_filter_dataset['groupColumnValue'].values[0]}"
                                        }
            # If column value as more than one item, add in figure. 
            else: 
                df_small = df_small.sort_values(by = df_pc_filter_dataset['unit'].iloc[0] , ascending=False)
                fig = px.bar(df_small, x = df_pc_filter_dataset['groupColumnName'].iloc[0],y =df_pc_filter_dataset['value'], template="plotly_white", height = 350 )
                fig.update_layout(
                    margin={'t':0,'l':0,'b':0,'r':0}
                )
                dataset_dict[dataset] = {"table":df_small.to_html(index = False),
                                     "fig":""#fig.to_html(full_html=False)
                                     }
    table_dict[category] = dataset_dict

Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Bedrock
Lakes Inventory Councils
Flood Alert Areas
Flood Warning Areas
Main Rivers
Risk of Flooding
Chalk Streams
Lakes Inventory Councils
Flood Alert Areas
Flood Warning Areas
Main Rivers
Risk of Flooding
Chalk Streams
Lakes Inventory Councils
Flood Alert Areas
Flood Warning Areas
Main Rivers
Risk of Flooding
Chalk Streams
Lakes Inventory Councils
Flood Alert Areas
Flood Warning Areas
Main Rivers
Risk of Flooding
Chalk Streams
Lakes Inventory Councils
Flood Alert Areas
Flood Warning Areas
Main Rivers
Risk of Flooding
Chalk Streams
National Forest Inventory
National F

In [138]:
environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("Stats template.html")


filename = f"outputReports/{pc}_stats.html"
# filename = f"outputReports/test_stats.html"

content = template.render(
    pc = pc,
    pcAreaHa = format(pcAreaHa.round(-1),","),
    pcAreaRank = pcAreaRank,
    scores = df_pc_filter,
    data = table_dict

)
with open(filename, mode="w", encoding="utf-8") as message:
    message.write(content)
    print(f"... wrote {filename}")

... wrote outputReports/Wroughton_stats.html
